In [1]:
import pandas as pd
import numpy as np
import copy
from datetime import datetime
import argparse
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score as auprc
from sklearn.metrics import roc_auc_score as auc_score
import keras
from keras.utils import multi_gpu_model
from keras.layers import Input, Dense, GRU, Lambda, Permute
from keras.models import Model
from interpolation_layer import single_channel_interp, cross_channel_interp
import warnings
from sklearn.preprocessing import KBinsDiscretizer
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
vitals_dict = {}
np.random.seed(10)
num_features = 5
max_length = 2000 # length of timestamp
gpu_num = 0
ref_points = 192
hid = 100
batch = 256
epoch = 100
num_ids = 200
hours_looks_ahead = 24

In [3]:
admission = pd.read_csv('Dataset/temp_admission.csv')
admission = admission[["HADM_ID", "ADMISSION_TYPE", "LOS"]]
admission['LOS'] = admission['LOS'] * 24
admission = admission[admission['LOS'] >= hours_looks_ahead]

admission.head()

,HADM_ID,ADMISSION_TYPE,LOS
0,165315,EMERGENCY,27.466667
1,152223,ELECTIVE,131.916667
2,124321,EMERGENCY,162.433333
3,161859,EMERGENCY,68.566667
4,129635,EMERGENCY,84.816667


In [4]:

# test_ = item220045_.loc[item220045_['HADM_ID'] == 152223].sort_values(by=['CHARTTIME'])
# test1 = test_.groupby('HADM_ID')['CHARTTIME'].apply(list).tolist()[0]
# test2 = test_.groupby('HADM_ID')['VALUENUM'].apply(list)
# # for i in test1:
# #     print(i)
# # for j in test2:
# #     print(j)
# test1,test2

In [5]:
# Heart Rate
item220045 = pd.read_csv('Dataset/temp220045.csv')
item220045.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item220045['CHARTTIME'] = pd.to_datetime(item220045['CHARTTIME']).dt.tz_localize(None)
item220045 = item220045[item220045['VALUENUM'].notna()]
item220045 = pd.merge(admission[['HADM_ID', 'LOS']],
                 item220045[['HADM_ID', 'CHARTTIME', 'VALUENUM']],
                 on='HADM_ID', 
                 how='inner')

item220045.head()

,HADM_ID,LOS,CHARTTIME,VALUENUM
0,165315,27.466667,2196-04-09 21:00:00,86.0
1,165315,27.466667,2196-04-10 06:00:00,105.0
2,165315,27.466667,2196-04-10 14:00:00,99.0
3,165315,27.466667,2196-04-09 23:00:00,92.0
4,165315,27.466667,2196-04-10 02:00:00,111.0


In [6]:
# Arterial Blood Pressure systolic
item220050 = pd.read_csv('Dataset/temp220050.csv')
item220050.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'ITEM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item220050 = item220050[item220050['VALUENUM'].notna()]
item220050['CHARTTIME'] = pd.to_datetime(item220050['CHARTTIME']).dt.tz_localize(None)
item220050 = pd.merge(admission[['HADM_ID', 'LOS']],
                 item220050[['HADM_ID', 'CHARTTIME', 'VALUENUM']],
                 on='HADM_ID', 
                 how='inner')
item220050.head()

,HADM_ID,LOS,CHARTTIME,VALUENUM
0,152223,131.916667,2153-09-04 14:00:00,122.0
1,152223,131.916667,2153-09-04 03:00:00,93.0
2,152223,131.916667,2153-09-03 22:15:00,84.0
3,152223,131.916667,2153-09-04 02:00:00,88.0
4,152223,131.916667,2153-09-04 04:30:00,110.0


In [7]:
# Non Invasive Blood Pressure systolic
item220179 = pd.read_csv('Dataset/temp220179.csv')
item220179.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item220179 = item220179[item220179['VALUENUM'].notna()]
item220179['CHARTTIME'] = pd.to_datetime(item220179['CHARTTIME']).dt.tz_localize(None)
item220179 = pd.merge(admission[['HADM_ID', 'LOS']],
                 item220179[['HADM_ID', 'CHARTTIME', 'VALUENUM']],
                 on='HADM_ID', 
                 how='inner')
item220179.head()

,HADM_ID,LOS,CHARTTIME,VALUENUM
0,165315,27.466667,2196-04-09 23:00:00,146.0
1,165315,27.466667,2196-04-10 00:00:00,139.0
2,165315,27.466667,2196-04-09 20:30:00,130.0
3,165315,27.466667,2196-04-10 02:30:00,138.0
4,165315,27.466667,2196-04-09 22:00:00,163.0


In [8]:
# Temperature Fahrenheit
item223761 = pd.read_csv('Dataset/temp223761.csv')
item223761.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item223761 = item223761[item223761['VALUENUM'].notna()]
item223761['CHARTTIME'] = pd.to_datetime(item223761['CHARTTIME']).dt.tz_localize(None)
item223761 = pd.merge(admission[['HADM_ID', 'LOS']],
                 item223761[['HADM_ID', 'CHARTTIME', 'VALUENUM']],
                 on='HADM_ID', 
                 how='inner')
item223761.head()

,HADM_ID,LOS,CHARTTIME,VALUENUM
0,165315,27.466667,2196-04-09 20:30:00,97.599998
1,165315,27.466667,2196-04-10 14:00:00,99.400002
2,165315,27.466667,2196-04-10 11:00:00,98.500000
3,152223,131.916667,2153-09-04 14:00:00,98.099998
4,152223,131.916667,2153-09-04 17:00:00,98.400002


In [9]:
# Temperature C
item676 = pd.read_csv('Dataset/temp676.csv')
item676['CHARTTIME'] = pd.to_datetime(item676['CHARTTIME']).dt.tz_localize(None)
item676 = item676[item676['VALUENUM'].notna()]
item676 = pd.merge(admission[['HADM_ID', 'LOS']],
                 item676[['HADM_ID', 'CHARTTIME', 'VALUENUM']],
                 on='HADM_ID', 
                 how='inner')
item676.head()

,HADM_ID,LOS,CHARTTIME,VALUENUM
0,152223,131.916667,2153-09-03 15:45:00,37.299999
1,152223,131.916667,2153-09-03 16:00:00,37.299999
2,152223,131.916667,2153-09-04 02:00:00,37.400002
3,152223,131.916667,2153-09-03 19:00:00,37.299999
4,152223,131.916667,2153-09-03 15:00:00,37.299999


In [10]:
# Inspired O2 Fraction (FiO2)
item223835 = pd.read_csv('Dataset/temp223835.csv')
item223835.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'ITEM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item223835 = item223835[item223835['VALUENUM'].notna()]
item223835['CHARTTIME'] = pd.to_datetime(item223835['CHARTTIME']).dt.tz_localize(None)
item223835 = pd.merge(admission[['HADM_ID', 'LOS']],
                 item223835[['HADM_ID', 'CHARTTIME', 'VALUENUM']],
                 on='HADM_ID', 
                 how='inner')
item223835.head()

,HADM_ID,LOS,CHARTTIME,VALUENUM
0,152223,131.916667,2153-09-03 22:00:00,0.5
1,152223,131.916667,2153-09-03 20:45:00,0.6
2,152223,131.916667,2153-09-04 00:00:00,0.4
3,152223,131.916667,2153-09-03 19:20:00,1.0
4,152223,131.916667,2153-09-03 23:30:00,0.5


In [11]:
# RR
item618 = pd.read_csv('Dataset/temp618.csv')
item618['CHARTTIME'] = pd.to_datetime(item618['CHARTTIME']).dt.tz_localize(None)
item618 = item618[item618['VALUENUM'].notna()]
item618 = pd.merge(admission[['HADM_ID', 'LOS']],
                 item618[['HADM_ID', 'CHARTTIME', 'VALUENUM']],
                 on='HADM_ID', 
                 how='inner')

item618.head()

,HADM_ID,LOS,CHARTTIME,VALUENUM
0,165315,27.466667,2196-04-10 01:00:00,22.0
1,165315,27.466667,2196-04-10 04:00:00,20.0
2,165315,27.466667,2196-04-09 21:00:00,18.0
3,152223,131.916667,2153-09-03 15:45:00,12.0
4,152223,131.916667,2153-09-03 16:00:00,12.0


In [12]:
# RR should be merged with 618
item220210 = pd.read_csv('Dataset/temp220210.csv')
item220210['CHARTTIME'] = pd.to_datetime(item220210['CHARTTIME']).dt.tz_localize(None)
item220210 = item220210[item220210['VALUENUM'].notna()]
item220210 = pd.merge(admission[['HADM_ID', 'LOS']],
                 item220210[['HADM_ID', 'CHARTTIME', 'VALUENUM']],
                 on='HADM_ID', 
                 how='inner')

item220210.head()

,HADM_ID,LOS,CHARTTIME,VALUENUM
0,124321,162.433333,2157-10-22 10:00:00,16.0
1,124321,162.433333,2157-10-22 11:00:00,19.0
2,124321,162.433333,2157-10-22 12:00:00,18.0
3,124321,162.433333,2157-10-22 13:00:00,18.0
4,124321,162.433333,2157-10-22 14:00:00,21.0


In [13]:
item_rr = item618.append(item220210, ignore_index=True)

In [14]:
# spo2 
item646 = pd.read_csv('Dataset/temp646.csv')
item646['CHARTTIME'] = pd.to_datetime(item646['CHARTTIME']).dt.tz_localize(None)
item646 = item646[item646['VALUENUM'].notna()]
item646 = pd.merge(admission[['HADM_ID', 'LOS']],
                 item646[['HADM_ID', 'CHARTTIME', 'VALUENUM']],
                 on='HADM_ID', 
                 how='inner')
item646.head()

,HADM_ID,LOS,CHARTTIME,VALUENUM
0,165315,27.466667,2196-04-10 09:00:00,99.0
1,165315,27.466667,2196-04-10 01:00:00,97.0
2,165315,27.466667,2196-04-10 04:00:00,97.0
3,165315,27.466667,2196-04-10 07:00:00,97.0
4,165315,27.466667,2196-04-09 13:30:00,100.0


In [15]:
# spo2 should be merged with 646
item220277 = pd.read_csv('Dataset/temp220277.csv')
item220277['CHARTTIME'] = pd.to_datetime(item220277['CHARTTIME']).dt.tz_localize(None)
item220277 = item220277[item220277['VALUENUM'].notna()]
item220277 = pd.merge(admission[['HADM_ID', 'LOS']],
                 item220277[['HADM_ID', 'CHARTTIME', 'VALUENUM']],
                 on='HADM_ID', 
                 how='inner')
item220277.head()

,HADM_ID,LOS,CHARTTIME,VALUENUM
0,124321,162.433333,2157-10-22 10:00:00,98.0
1,124321,162.433333,2157-10-22 11:00:00,97.0
2,124321,162.433333,2157-10-22 12:00:00,98.0
3,124321,162.433333,2157-10-22 13:00:00,96.0
4,124321,162.433333,2157-10-22 14:00:00,98.0


In [16]:
item_spo2 = item646.append(item220277, ignore_index=True)

In [17]:
# DBP
item8441 = pd.read_csv('Dataset/temp8441.csv')
item8441['CHARTTIME'] = pd.to_datetime(item8441['CHARTTIME']).dt.tz_localize(None)
item8441 = item8441[item8441['VALUENUM'].notna()]
item8441 = pd.merge(admission[['HADM_ID', 'LOS']],
                 item8441[['HADM_ID', 'CHARTTIME', 'VALUENUM']],
                 on='HADM_ID', 
                 how='inner')

item8441.head()

,HADM_ID,LOS,CHARTTIME,VALUENUM
0,165315,27.466667,2196-04-09 13:30:00,65.0
1,165315,27.466667,2196-04-09 14:00:00,65.0
2,165315,27.466667,2196-04-09 15:00:00,91.0
3,165315,27.466667,2196-04-09 16:00:00,73.0
4,165315,27.466667,2196-04-09 17:00:00,72.0


In [18]:
# return two lists of hadm_id: list of timestamps (ascending order), list of corresponding vals of the item
def time_val_toLst(df, hadmin_id):
    temp = df.loc[df['HADM_ID'] == hadmin_id].sort_values(by=['CHARTTIME'])
    if not temp.empty: 
        times = temp.groupby('HADM_ID')['CHARTTIME'].apply(list).tolist()[0]
        vals = temp.groupby('HADM_ID')['VALUENUM'].apply(list).tolist()[0]
        return [times, vals]
    else:
        return [] # if id does not have this item, return empty list

In [19]:
def flatten(df, los=hours_looks_ahead):
    a = np.full((len(df), num_features, max_length), -100) # initlizae all as missing
    timestamps = []
    for i in range(len(df)):
        l = []
    
        # find all the unique observed timestamps
        for j in range(num_features):
            ts_ij = df[i][j]
            if ts_ij != []:
                for ts in df[i][j][0]:
                    if ts not in l:
                        l.append(ts)
        l.sort()
        T = copy.deepcopy(l)
        TS = []
        for t in T:
            if (t - T[0]).total_seconds() / 3600 <= los:
                TS.append(t)
        timestamps.append(TS)
        for j in range(num_features):
            s_ij = df[i][j]
            if s_ij != []:
                ts_ij = s_ij[0] 
                c_max = len(ts_ij)
                c = 0
                for k in range(len(TS)):
                    if c < c_max:
                        ts_ijc = ts_ij[c] # cur ts
                        diff = abs(TS[k] - ts_ijc).seconds / 60 # difference between ts's
                        if TS[k] == ts_ijc or diff < 5:
                            a[i,j,k] = s_ij[1][c] 
                            c += 1
    return a, timestamps

In [20]:
def input_format(x, T):
    for lst in T:
        if len(lst) > max_length:
            T[i] = lst[:max_length]

    x = a[:, :, :max_length]
    M = np.zeros_like(x)
    delta = np.zeros_like(x)

    for t in T:
        for i in range(1, len(t)):
            t[i] = (t[i] - t[0]).total_seconds() / 3600 # hours difference
        if len(t) != 0:
            t[0] = 0
    
    # count outliers and negative values as missing values
    # M = 0 indicates missing value
    # M = 1 indicates observed value
    # now since we have mask variable, we don't need -100
    M[x > 500] = 0
    x[x > 500] = 0.0
    M[x < 0] = 0
    x[x < 0] = 0.0
    M[x > 0] = 1

    for i in range(num_features):
        for j in range(x.shape[0]):
            for k in range(len(T[j])):
                delta[j, i, k] = T[j][k]
    return x, M, delta

In [21]:
def missing_mean(M, x):
    counts = np.sum(np.sum(M, axis=2), axis=0)
    mean_values = np.sum(np.sum(x*M, axis=2), axis=0)/counts
    for i in range(M.shape[0]):
        for j in range(M.shape[1]):
            if np.sum(M[i, j]) == 0:
                M[i, j, 0] = 1
                x[i, j, 0] = mean_values[j]
    return 

In [22]:
def hold_out(mask, perc=0.2):
    """To implement the autoencoder component of the loss, we introduce a set
    of masking variables mr (and mr1) for each data point. If drop_mask = 0,
    then we removecthe data point as an input to the interpolation network,
    and includecthe predicted value at this time point when assessing
    the autoencoder loss. In practice, we randomly select 20% of the
    observed data points to hold out from
    every input time series."""
    drop_mask = np.ones_like(mask)
    drop_mask *= mask
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            count = np.sum(mask[i, j], dtype='int')
            if int(0.20*count) > 1:
                index = 0
                r = np.ones((count, 1))
                b = np.random.choice(count, int(0.20*count), replace=False)
                r[b] = 0
                for k in range(mask.shape[2]):
                    if mask[i, j, k] > 0:
                        drop_mask[i, j, k] = r[index]
                        index += 1
    return drop_mask

In [23]:
# create the vitals dictionary: {id: [item1, item2, item3, item4, item5]}
# item1 = [list of timestamps, list of vals]
# only use the first 20 hadm_ids as sample

adm_ids = admission['HADM_ID'].tolist()
for adm_id in adm_ids[:num_ids]:
    vitals_dict[adm_id] = [time_val_toLst(item220045, adm_id)]
    vitals_dict[adm_id].append(time_val_toLst(item220050, adm_id))
    vitals_dict[adm_id].append(time_val_toLst(item220179, adm_id))
#     vitals_dict[adm_id].append(time_val_toLst(item223761, adm_id))
    vitals_dict[adm_id].append(time_val_toLst(item676, adm_id))
    vitals_dict[adm_id].append(time_val_toLst(item223835, adm_id))
#     vitals_dict[adm_id].append(time_val_toLst(item_rr, adm_id))
#     vitals_dict[adm_id].append(time_val_toLst(item_spo2, adm_id))
#     vitals_dict[adm_id].append(time_val_toLst(item8441, adm_id))

In [24]:
vitals = [vitals_dict[x] for x in adm_ids[:num_ids]] # hadm_id(los>=48h): all the vitals values
kb = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='kmeans')
label = kb.fit_transform(admission['LOS'][:num_ids].to_numpy().reshape(-1,1))
print(kb.bin_edges_ / 24)

[array([  1.01527778,  38.40479814,  79.83046875, 107.60486111])]


In [25]:
a, ts = flatten(vitals, hours_looks_ahead)
x, m, T = input_format(a, ts)
missing_mean(m, x)
X = np.concatenate((x, m, T, hold_out(m)), axis=1)  # input format
y = np.array(label)
print(X.shape, y.shape)
timestamp = X.shape[2]
num_features = X.shape[1] // 4

(200, 20, 2000) (200, 1)


In [26]:
num_features

5

In [27]:
# 5.7, 3.33, 14.1
def customloss(ytrue, ypred):
    """ Autoencoder loss
    """
    num_fs = num_features
    # standard deviation of each feature mentioned in paper for MIMIC_III data
    wc = np.array([17.3, 22, 22.45, 2.32, 23.41])
    print(wc.shape, ytrue.shape)
    wc.shape = (1, num_fs)
    y = ytrue[:, :num_fs, :]
    m2 = ytrue[:, 3*num_fs:4*num_fs, :]
    m2 = 1 - m2
    m1 = ytrue[:, num_fs:2*num_fs, :]
    m = m1*m2
    ypred = ypred[:, :num_fs, :]
    x = (y - ypred)*(y - ypred)
    x = x*m
    count = tf.reduce_sum(m, axis=2)
    count = tf.where(count > 0, count, tf.ones_like(count))
    x = tf.reduce_sum(x, axis=2)/count
    x = x/(wc**2)  # dividing by standard deviation
    x = tf.reduce_sum(x, axis=1)/num_fs
    return tf.reduce_mean(x)

In [31]:
def interp_net():
    num_fs = num_features
    if gpu_num > 1:
        dev = "/cpu:0"
    else:
        dev = "/gpu:0"
    with tf.device(dev):
        main_input = Input(shape=(4*num_fs, timestamp), name='input')
        sci = single_channel_interp(ref_points, hours_looks_ahead)
        cci = cross_channel_interp()
        interp = cci(sci(main_input))
        reconst = cci(sci(main_input, reconstruction=True),
                      reconstruction=True)
        aux_output = Lambda(lambda x: x, name='aux_output')(reconst)
        z = Permute((2, 1))(interp)
        z = GRU(hid, activation='tanh', recurrent_dropout=0.2, dropout=0.2)(z)
        main_output = Dense(1, activation='sigmoid', name='main_output')(z)
        orig_model = Model([main_input], [main_output, aux_output])
    if gpu_num > 1:
        model = multi_gpu_model(orig_model, gpus=gpu_num)
    else:
        model = orig_model
    print(orig_model.summary())
    return model

In [32]:
seed = 0
results = {}
results['loss'] = []
results['auc'] = []
results['acc'] = []
results['auprc'] = []
earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0000, patience=20, verbose=0)
callbacks_list = [earlystop]


In [ ]:
# 5-fold cross-validation
i = 0
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
for train, test in kfold.split(np.zeros(len(y)), y):
    print("Running Fold:", i+1)
    model = interp_net()  # re-initializing every time
    model.compile(
        optimizer='adam',
        loss={'main_output': 'binary_crossentropy', 'aux_output': customloss},
        loss_weights={'main_output': 1., 'aux_output': 1.},
        metrics={'main_output': 'accuracy'})
    model.fit(
        {'input': X[train]}, {'main_output': y[train], 'aux_output': X[train]},
        batch_size=batch,
        callbacks=callbacks_list,
        nb_epoch=epoch,
        validation_split=0.20,
        verbose=2)
    y_pred = model.predict(X[test], batch_size=batch)
    y_pred = y_pred[0]
    total_loss, score, reconst_loss, acc = model.evaluate(
        {'input': X[test]},
        {'main_output': y[test], 'aux_output': X[test]},
        batch_size=batch,
        verbose=0)
    results['loss'].append(score)
    results['acc'].append(acc)
    results['auc'].append(auc_score(y[test], y_pred))
    results['auprc'].append(auprc(y[test], y_pred))
    print(results)
    i += 1

Running Fold: 1
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 20, 2000)     0                                            
__________________________________________________________________________________________________
single_channel_interp_1 (single multiple             5           input[0][0]                      
                                                                 input[0][0]                      
__________________________________________________________________________________________________
cross_channel_interp_1 (cross_c multiple             25          single_channel_interp_1[0][0]    
                                                                 single_channel_interp_1[1][0]    
____________________________________________________________________________

In [404]:
num_features 

20

In [296]:
# for lst in timestamps:
#     if len(lst) > max_length:
#         timestamps[i] = lst[:max_length]

# x = a[:, :, :max_length]
# M = np.zeros_like(x)
# delta = np.zeros_like(x)

# for t in timestamps:
#     for i in range(1, len(t)):
#         t[i] = (t[i] - t[0]).total_seconds() / 3600 # hours difference
#     if len(t) != 0:
#         t[0] = 0

In [220]:
# a = np.full((len(vitals), num_features, max_length), -100) # initlizae all as missing
# timestamps = []
# for i in range(len(vitals)):
#     l = []
    
#     # find all the unique observed timestamps
#     for j in range(num_features):
#         ts_ij = vitals[i][j]
#         if ts_ij != []:
#             for ts in vitals[i][j][0]:
#                 if ts not in l:
#                     l.append(ts)
#     l.sort()
#     T = copy.deepcopy(l)
#     TS = []
#     for t in T:
#         if (t - T[0]).total_seconds() / 3600 <= 72:
#             TS.append(t)
#     timestamps.append(TS)
#     for j in range(num_features):
#         s_ij = vitals[i][j]
#         if s_ij != []:
#             ts_ij = s_ij[0] 
#             c_max = len(ts_ij)
#             c = 0
#             for k in range(len(TS)):
#                 if c < c_max:
#                     ts_ijc = ts_ij[c] # cur ts
#                     diff = abs(TS[k] - ts_ijc).seconds / 60 # difference between ts's
#                     if TS[k] == ts_ijc or diff < 5:
#                         a[i,j,k] = s_ij[1][c] 
#                         c += 1

In [238]:
# # count outliers and negative values as missing values
# # M = 0 indicates missing value
# # M = 1 indicates observed value
# # now since we have mask variable, we don't need -100
# M[x > 500] = 0
# x[x > 500] = 0.0
# M[x < 0] = 0
# x[x < 0] = 0.0
# M[x > 0] = 1

# for i in range(num_features):
#         for j in range(x.shape[0]):
#             for k in range(len(timestamps[j])):
#                 delta[j, i, k] = timestamps[j][k]

In [185]:
x = np.concatenate((x, m, T, hold_out(m)), axis=1)  # input format
y = np.array(label)
print(x.shape, y.shape)
timestamp = x.shape[2]
num_features = x.shape[1] // 4

[39, 38, 13, 33, 22, 2, 37, 21, 29, 32, 27, 31, 53, 35, 47, 30, 31, 44, 25, 28]